In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
### 一、数据预处理（仅做必要处理，后文模型中非核心变量的处理见数据处理）

# Step 1:数据读取
file_path = 'D:/期中考试/input/quant4533/'
train_data = pd.read_csv(file_path+'ruc_Class25Q1_train.csv')
test_data = pd.read_csv(file_path+'ruc_Class25Q1_test.csv')
details_data=pd.read_csv(file_path+'ruc_Class25Q1_details.csv')

# Step 2:修改变量
columns_to_process = ['建筑面积', '套内面积']
for column in columns_to_process:
    # 删除“㎡”并转换为空格分隔的数字
    train_data[column] = train_data[column].str.replace('㎡', '', regex=False)
    test_data[column] = test_data[column].str.replace('㎡', '', regex=False)
    # 去除可能存在的多余空格
    train_data[column] = train_data[column].str.strip()
    test_data[column] = test_data[column].str.strip()
    # 将清理后的字符串转换为浮点数类型
    train_data[column] = pd.to_numeric(train_data[column], errors='coerce')
    test_data[column] = pd.to_numeric(test_data[column], errors='coerce')

details_data['建筑年代'] = details_data['建筑年代'].str.replace('年', '')
details_data['房屋总数'] = details_data['房屋总数'].str.replace('户', '').astype(int)
details_data['楼栋总数'] = details_data['楼栋总数'].str.replace('栋', '').astype(int)
details_data['绿化率'] = details_data['绿 化 率'].str.replace('%', '').astype(float)
details_data=details_data.drop(columns='绿 化 率')
details_data['物业费'] = details_data['物 业 费'].str.replace('元/月/㎡', '')
details_data=details_data.drop(columns='物 业 费')
details_data['燃气费'] = details_data['燃气费'].str.replace('元/m³', '')
details_data['供热费'] = details_data['供热费'].str.replace('元/㎡', '')

# Step 3:删除变量（将本模型不计划使用的均剔除，减小内存负担）
columns1_to_drop = ['交易时间', '抵押信息', '上次交易', '核心卖点', '户型介绍', '周边配套', '交通出行']
train_data = train_data.drop(columns=columns1_to_drop)
test_data = test_data.drop(columns=columns1_to_drop)
#columns2_to_drop = ['区县', '城市', '板块', '环线位置', '小区地址', '物业类别','物业公司','开发商','建筑结构','物业办公电话','产权描述','停车费用','coord_x','coord_y']
columns2_to_drop = ['区县', '城市',  '环线位置', '小区地址', '物业类别','物业公司','开发商','建筑结构','物业办公电话','产权描述','停车费用','coord_x','coord_y']
details_data=details_data.drop(columns=columns2_to_drop)

# Step 4:填充数据（将所有缺失值替换为 0）
train_data = train_data.fillna(0)
test_data = test_data.fillna(0)
details_data=details_data.fillna(0)


#保存
train_data.to_excel("D:/期中考试/"+"project/train_v1.xlsx", index=False)
test_data.to_excel("D:/期中考试/"+"project/test_v1.xlsx", index=False)
details_data.to_excel("D:/期中考试/"+"project/details_v1.xlsx", index=False)

In [ ]:
###二、训练及效果评估

# Step 1: 数据读取
data_path = "D:/期中考试/project/train_v1.xlsx"
data = pd.read_excel(data_path)
details_data=pd.read_excel("D:/期中考试/project/details_v1.xlsx")

# Step 2: 数据处理
# Step 2.1: 保留核心变量
combined_data = data[['ID','价格', '城市', '区域','板块','小区名称','环线', '所在楼层','房屋户型', '梯户比例', 
'配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限','建筑面积', '套内面积','房屋朝向','is_train','lon','lat']]
details_data=pd.read_excel("D:/期中考试/project/details_v1.xlsx")
details_data=details_data[['板块','名称','容 积 率','绿化率']]

# 根据小区名称和名称进行匹配（左连接）
combined_data = combined_data.merge(details_data, left_on=['板块','小区名称'], right_on=['板块','名称'], how='left')
combined_data.drop(columns=['名称'], inplace=True) 
combined_data['小区名称'] = combined_data['小区名称'] + combined_data['板块'].astype(str)

# Step 2.2: 生成新变量-对数化面积（加1是为了克服部分为0的数据）、把小区变量进行变换，克服共线性问题
#combined_data['价格']=combined_data['价格']/combined_data['建筑面积']
combined_data['套内面积'] = combined_data['套内面积'].replace(0, combined_data['套内面积'].mean())
combined_data['套内面积比'] = combined_data['套内面积'] / combined_data['建筑面积']
combined_data['建筑面积_对数'] = np.log(combined_data['建筑面积'] + 1)
combined_data['套内面积_对数'] = np.log(combined_data['套内面积'] + 1)
combined_data['建筑面积_平方'] = combined_data['建筑面积'] ** 2
combined_data['套内面积_平方'] = combined_data['套内面积'] ** 2
combined_data['绿化率']=combined_data['绿化率']*combined_data['建筑面积']/100
combined_data['容 积 率']=combined_data['容 积 率']*combined_data['建筑面积']
combined_data['面积级别'] = (combined_data['建筑面积'] / 35 + 1).astype(int)

# Step 2.3:转换变量-房屋户型（分为5个变量）
combined_data['室'] = combined_data['房屋户型'].str.extract(r'(\d+)室').astype(float)
combined_data['厅'] = combined_data['房屋户型'].str.extract(r'(\d+)厅').astype(float)
combined_data['厨'] = combined_data['房屋户型'].str.extract(r'(\d+)厨').astype(float)
combined_data['卫'] = combined_data['房屋户型'].str.extract(r'(\d+)卫').astype(float)
combined_data['房间'] = combined_data['房屋户型'].str.extract(r'(\d+)房间').astype(float)
combined_data=combined_data.fillna(0)
combined_data = combined_data.drop('房屋户型', axis=1)
#转为int，便于后续根据是否为0来进行填充
combined_data['室'] = combined_data['室'].astype(int)
combined_data['厅'] = combined_data['厅'].astype(int)
combined_data['厨'] = combined_data['厨'].astype(int)
combined_data['卫'] = combined_data['卫'].astype(int).replace(0, 1)
combined_data['房间'] = combined_data['房间'].astype(int)

# 如果房间数目为 0，则将室和厅的值相加填入房间
combined_data['房间'] = combined_data.apply(lambda row: row['室'] + row['厅'] if row['房间'] == 0 else row['房间'], axis=1)
combined_data['室卫比'] = combined_data['房间'] / combined_data['卫'].replace(0, 1)

# Step 2.4: 转换变量-所在楼层（分为楼层类型和总层数）
def extract_floor_info(floor_str):
    # 提取楼层类型（如“中楼层”）
    floor_type = re.match(r'(.+) \(', floor_str).group(1)
    # 提取总层数（如“25”）
    total_floors = int(re.search(r'共(\d+)层', floor_str).group(1))
    return floor_type, total_floors
combined_data['楼层类型'] = combined_data['所在楼层'].apply(lambda x: extract_floor_info(x)[0])
combined_data['总层数'] = combined_data['所在楼层'].apply(lambda x: extract_floor_info(x)[1])
combined_data.drop(columns=['所在楼层'], inplace=True)
# Step 2.5: 转换变量-梯户比例（从"X梯X户"转换为数值型变量）
def convert_ratio(ratio_str):
    # 汉字数字映射
    chinese_to_num = {
    '一': 1, '二': 2, '两': 2, '三': 3, '四': 4, '五': 5,'六': 6, '七': 7, '八': 8, '九': 9, '十': 10,
    '十一': 11, '十二': 12, '十三': 13, '十四': 14, '十五': 15,'十六': 16, '十七': 17, '十八': 18, '十九': 19, '二十': 20,
    '二十一': 21, '二十二': 22, '二十三': 23, '二十四': 24, '二十五': 25,'二十六': 26, '二十七': 27, '二十八': 28, '二十九': 29, '三十': 30,
    '三十一': 31, '三十二': 32, '三十三': 33, '三十四': 34, '三十五': 35,'三十六': 36, '三十七': 37, '三十八': 38, '三十九': 39, '四十': 40,
    '四十一': 41, '四十二': 42, '四十三': 43, '四十四': 44, '四十五': 45,'四十六': 46, '四十七': 47, '四十八': 48, '四十九': 49, '五十': 50,
    '五十一': 51, '五十二': 52, '五十三': 53, '五十四': 54, '五十五': 55,'五十六': 56, '五十七': 57, '五十八': 58, '五十九': 59, '六十': 60,
    '六十一': 61, '六十二': 62, '六十三': 63, '六十四': 64, '六十五': 65,'六十六': 66, '六十七': 67, '六十八': 68, '六十九': 69, '七十': 70,
    '七十一': 71, '七十二': 72, '七十三': 73, '七十四': 74, '七十五': 75,'七十六': 76, '七十七': 77, '七十八': 78, '七十九': 79, '八十': 80,
    '八十一': 81, '八十二': 82, '八十三': 83, '八十四': 84, '八十五': 85,'八十六': 86, '八十七': 87, '八十八': 88, '八十九': 89, '九十': 90,
    '九十一': 91, '九十二': 92, '九十三': 93, '九十四': 94, '九十五': 95,'九十六': 96, '九十七': 97, '九十八': 98, '九十九': 99, '一百': 100
    }
     # 如果输入是 '0'，直接返回 0
    if ratio_str == '0':
        return 0
    # 提取梯数和户数
    parts = ratio_str.split('梯')
    if len(parts) != 2:
        raise ValueError(f"Invalid format: {ratio_str}")
    elevators = parts[0].strip()
    households = parts[1].replace('户', '').strip()
    # 转换为阿拉伯数字
    elevators_num = chinese_to_num.get(elevators, elevators)
    households_num = chinese_to_num.get(households, households)
    # 计算比例
    try:
        ratio = int(households_num) / int(elevators_num)
    except ZeroDivisionError:
        ratio = np.nan  # 如果梯数为0，返回NaN
    return ratio
combined_data['梯户比例'] = combined_data['梯户比例'].astype(str)
combined_data['梯户比例'] = combined_data['梯户比例'].apply(convert_ratio)


# Step 2.6: 生成新变量-虚拟变量（本质为固定效应）
combined_data = pd.get_dummies(combined_data, columns=['城市', '区域','板块','小区名称','环线', '楼层类型','配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限','面积级别'], drop_first=True)


# Step 2.7:生成新变量-小区*面积的交乘项（主因在于，小区本身对应的虚拟变量有3000+个，若直接将其通过polynomial函数处理，会导致内存不足）
neighborhood_columns = [col for col in combined_data.columns if col.startswith('板块_')]
for col in neighborhood_columns:
    # 与建筑面积的交乘项
    new_col_name = f'{col}_建筑面积'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积']

    new_col_name = f'{col}_建筑面积_对数'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积_对数']

    new_col_name = f'{col}_建筑面积_平方'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积_平方']

# Step 2.8:基本朝向
combined_data['朝南'] = combined_data['房屋朝向'].str.contains('南', na=False).astype(int)
combined_data['朝北'] = combined_data['房屋朝向'].str.contains('北', na=False).astype(int)
combined_data['朝东'] = combined_data['房屋朝向'].str.contains('东', na=False).astype(int)
combined_data['朝西'] = combined_data['房屋朝向'].str.contains('西', na=False).astype(int)
combined_data=combined_data.drop(columns=['房屋朝向'])
# 复合特征
combined_data['朝向数量'] = combined_data[['朝南', '朝北', '朝东', '朝西']].sum(axis=1)
combined_data['南北通透'] = ((combined_data['朝南'] == 1) & (combined_data['朝北'] == 1)).astype(int)
combined_data['到市中心距离'] = np.sqrt((combined_data['lon'] - 116.4) ** 2 + (combined_data['lat'] - 39.9) ** 2)
combined_data=combined_data.drop(columns=['lon','lat'])

 


# Step 3: 划分训练集和测试集
X = combined_data.drop(columns=['价格'])  # 自变量
y = combined_data['价格']  # 因变量

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=111
)

# Step 4: 构建回归模型
# 定义模型列表
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(alpha=0.1),
    "Ridge": Ridge(alpha=1000),
    "ElasticNet": ElasticNet(alpha=0.1, l1_ratio=0.5)
}

# 定义性能评估函数
def evaluate_model(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    in_sample_mae = mean_absolute_error(y_train, y_train_pred)
    out_of_sample_mae = mean_absolute_error(y_test, y_test_pred)
    
    return in_sample_mae, out_of_sample_mae

# Step 5: 6 折交叉验证
kf = KFold(n_splits=6, shuffle=True, random_state=111)

results = []
for name, model in models.items():
    # 交叉验证性能
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_mean_absolute_error')
    cv_mae = -cv_scores.mean()
    
    # 样本内和样本外性能
    in_sample_mae, out_of_sample_mae = evaluate_model(model, X_train, X_test, y_train, y_test)
    
    results.append({
        "Model": name,
        "In-Sample MAE": in_sample_mae,
        "Out-of-Sample MAE": out_of_sample_mae,
        "6-Fold CV MAE": cv_mae
    })

# 将结果转换为 DataFrame
results_df = pd.DataFrame(results)

# 打印结果
print("Model PerformancD:")
print(results_df)

In [ ]:
###Lasso
# Step 1: 使用 pandas 读取 excel 文件
train_data = pd.read_excel("D:/期中考试/project/train_v1.xlsx") 
test_data = pd.read_excel("D:/期中考试/project/test_v1.xlsx") 
details_data=pd.read_excel("D:/期中考试/project/details_v1.xlsx")
# Step 2: 数据处理
# Step 2.1: 保留核心变量
train_data['is_train'] = 1  # 标记训练集
test_data['is_train'] = 0   # 标记测试集
combined_data = pd.concat([train_data, test_data], axis=0)
combined_data = combined_data[['ID','价格', '城市', '区域','板块','小区名称','环线', '所在楼层','房屋户型', '梯户比例', 
'配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限','建筑面积', '套内面积','房屋朝向','is_train','lon','lat']]
details_data=pd.read_excel("D:/期中考试/project/details_v1.xlsx")
details_data=details_data[['板块','名称','容 积 率','绿化率']]

# 根据小区名称和名称进行匹配（左连接）
combined_data = combined_data.merge(details_data, left_on=['板块','小区名称'], right_on=['板块','名称'], how='left')
combined_data.drop(columns=['名称'], inplace=True) 
combined_data['小区名称'] = combined_data['小区名称'] + combined_data['板块'].astype(str)

# Step 2.2: 生成新变量-对数化面积（加1是为了克服部分为0的数据）、把小区变量进行变换，克服共线性问题
#combined_data['价格']=combined_data['价格']/combined_data['建筑面积']
combined_data['套内面积'] = combined_data['套内面积'].replace(0, combined_data['套内面积'].mean())
combined_data['套内面积比'] = combined_data['套内面积'] / combined_data['建筑面积']
combined_data['建筑面积_对数'] = np.log(combined_data['建筑面积'] + 1)
combined_data['套内面积_对数'] = np.log(combined_data['套内面积'] + 1)
combined_data['建筑面积_平方'] = combined_data['建筑面积'] ** 2
combined_data['套内面积_平方'] = combined_data['套内面积'] ** 2
combined_data['绿化率']=combined_data['绿化率']*combined_data['建筑面积']/100
combined_data['容 积 率']=combined_data['容 积 率']*combined_data['建筑面积']
combined_data['面积级别'] = (combined_data['建筑面积'] / 35 + 1).astype(int)

# Step 2.3:转换变量-房屋户型（分为5个变量）
combined_data['室'] = combined_data['房屋户型'].str.extract(r'(\d+)室').astype(float)
combined_data['厅'] = combined_data['房屋户型'].str.extract(r'(\d+)厅').astype(float)
combined_data['厨'] = combined_data['房屋户型'].str.extract(r'(\d+)厨').astype(float)
combined_data['卫'] = combined_data['房屋户型'].str.extract(r'(\d+)卫').astype(float)
combined_data['房间'] = combined_data['房屋户型'].str.extract(r'(\d+)房间').astype(float)
combined_data=combined_data.fillna(0)
combined_data = combined_data.drop('房屋户型', axis=1)
#转为int，便于后续根据是否为0来进行填充
combined_data['室'] = combined_data['室'].astype(int)
combined_data['厅'] = combined_data['厅'].astype(int)
combined_data['厨'] = combined_data['厨'].astype(int)
combined_data['卫'] = combined_data['卫'].astype(int).replace(0, 1)
combined_data['房间'] = combined_data['房间'].astype(int)

# 如果房间数目为 0，则将室和厅的值相加填入房间
combined_data['房间'] = combined_data.apply(lambda row: row['室'] + row['厅'] if row['房间'] == 0 else row['房间'], axis=1)
combined_data['室卫比'] = combined_data['房间'] / combined_data['卫'].replace(0, 1)

# Step 2.4: 转换变量-所在楼层（分为楼层类型和总层数）
def extract_floor_info(floor_str):
    # 提取楼层类型（如“中楼层”）
    floor_type = re.match(r'(.+) \(', floor_str).group(1)
    # 提取总层数（如“25”）
    total_floors = int(re.search(r'共(\d+)层', floor_str).group(1))
    return floor_type, total_floors
combined_data['楼层类型'] = combined_data['所在楼层'].apply(lambda x: extract_floor_info(x)[0])
combined_data['总层数'] = combined_data['所在楼层'].apply(lambda x: extract_floor_info(x)[1])
combined_data.drop(columns=['所在楼层'], inplace=True)
# Step 2.5: 转换变量-梯户比例（从"X梯X户"转换为数值型变量）
def convert_ratio(ratio_str):
    # 汉字数字映射
    chinese_to_num = {
    '一': 1, '二': 2, '两': 2, '三': 3, '四': 4, '五': 5,'六': 6, '七': 7, '八': 8, '九': 9, '十': 10,
    '十一': 11, '十二': 12, '十三': 13, '十四': 14, '十五': 15,'十六': 16, '十七': 17, '十八': 18, '十九': 19, '二十': 20,
    '二十一': 21, '二十二': 22, '二十三': 23, '二十四': 24, '二十五': 25,'二十六': 26, '二十七': 27, '二十八': 28, '二十九': 29, '三十': 30,
    '三十一': 31, '三十二': 32, '三十三': 33, '三十四': 34, '三十五': 35,'三十六': 36, '三十七': 37, '三十八': 38, '三十九': 39, '四十': 40,
    '四十一': 41, '四十二': 42, '四十三': 43, '四十四': 44, '四十五': 45,'四十六': 46, '四十七': 47, '四十八': 48, '四十九': 49, '五十': 50,
    '五十一': 51, '五十二': 52, '五十三': 53, '五十四': 54, '五十五': 55,'五十六': 56, '五十七': 57, '五十八': 58, '五十九': 59, '六十': 60,
    '六十一': 61, '六十二': 62, '六十三': 63, '六十四': 64, '六十五': 65,'六十六': 66, '六十七': 67, '六十八': 68, '六十九': 69, '七十': 70,
    '七十一': 71, '七十二': 72, '七十三': 73, '七十四': 74, '七十五': 75,'七十六': 76, '七十七': 77, '七十八': 78, '七十九': 79, '八十': 80,
    '八十一': 81, '八十二': 82, '八十三': 83, '八十四': 84, '八十五': 85,'八十六': 86, '八十七': 87, '八十八': 88, '八十九': 89, '九十': 90,
    '九十一': 91, '九十二': 92, '九十三': 93, '九十四': 94, '九十五': 95,'九十六': 96, '九十七': 97, '九十八': 98, '九十九': 99, '一百': 100
    }
     # 如果输入是 '0'，直接返回 0
    if ratio_str == '0':
        return 0
    # 提取梯数和户数
    parts = ratio_str.split('梯')
    if len(parts) != 2:
        raise ValueError(f"Invalid format: {ratio_str}")
    elevators = parts[0].strip()
    households = parts[1].replace('户', '').strip()
    # 转换为阿拉伯数字
    elevators_num = chinese_to_num.get(elevators, elevators)
    households_num = chinese_to_num.get(households, households)
    # 计算比例
    try:
        ratio = int(households_num) / int(elevators_num)
    except ZeroDivisionError:
        ratio = np.nan  # 如果梯数为0，返回NaN
    return ratio
combined_data['梯户比例'] = combined_data['梯户比例'].astype(str)
combined_data['梯户比例'] = combined_data['梯户比例'].apply(convert_ratio)


# Step 2.6: 生成新变量-虚拟变量（本质为固定效应）
#combined_data = pd.get_dummies(combined_data, columns=['城市', '区域','板块','小区名称','环线', '楼层类型','配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限'], drop_first=True)
combined_data = pd.get_dummies(combined_data, columns=['城市', '区域','板块','小区名称','环线', '楼层类型','配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限','面积级别'], drop_first=True)


# Step 2.7:生成新变量-小区*面积的交乘项（主因在于，小区本身对应的虚拟变量有3000+个，若直接将其通过polynomial函数处理，会导致内存不足）
neighborhood_columns = [col for col in combined_data.columns if col.startswith('板块_')]
for col in neighborhood_columns:
    # 与建筑面积的交乘项
    new_col_name = f'{col}_建筑面积'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积']

    new_col_name = f'{col}_建筑面积_对数'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积_对数']

    new_col_name = f'{col}_建筑面积_平方'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积_平方']

# Step 2.8:基本朝向
combined_data['朝南'] = combined_data['房屋朝向'].str.contains('南', na=False).astype(int)
combined_data['朝北'] = combined_data['房屋朝向'].str.contains('北', na=False).astype(int)
combined_data['朝东'] = combined_data['房屋朝向'].str.contains('东', na=False).astype(int)
combined_data['朝西'] = combined_data['房屋朝向'].str.contains('西', na=False).astype(int)
combined_data=combined_data.drop(columns=['房屋朝向'])
# 复合特征
combined_data['朝向数量'] = combined_data[['朝南', '朝北', '朝东', '朝西']].sum(axis=1)
combined_data['南北通透'] = ((combined_data['朝南'] == 1) & (combined_data['朝北'] == 1)).astype(int)
combined_data['到市中心距离'] = np.sqrt((combined_data['lon'] - 116.4) ** 2 + (combined_data['lat'] - 39.9) ** 2)
combined_data=combined_data.drop(columns=['lon','lat'])

# Step 3: 划分训练集和验证集
train_data = combined_data[combined_data['is_train'] == 1].drop(columns=['is_train'])
test_data = combined_data[combined_data['is_train'] == 0].drop(columns=['is_train'])
X_train = train_data.drop(columns=['价格','ID'])  # 自变量
y_train = train_data['价格']  # 因变量
# 按照训练集的列顺序对齐测试集
test_data = test_data[X_train.columns]


# Step 4: 创建 ColumnTransformer 来处理特征
non_poly_features = [col for col in X_train.columns if (col.startswith('板块_') or col.startswith('小区名称_') or col.startswith('面积级别'))]
poly_features = [col for col in X_train.columns if not (col.startswith('板块_') or col.startswith('小区名称_') or col.startswith('面积级别'))]

class CustomPolynomialFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2):
        self.degree = degree

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # 获取所有列名
        columns = X.columns.tolist()
        # 初始化交互项列表
        interaction_terms = []
        # 遍历所有列名，生成交互项
        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                # 检查是否属于不同类别
                if columns[i][:2] != columns[j][:2]:
                    interaction_terms.append((columns[i], columns[j]))
        # 生成交互项数据
        interaction_data = pd.DataFrame()
        for term in interaction_terms:
            interaction_data[f'{term[0]}_x_{term[1]}'] = X[term[0]] * X[term[1]]
        # 返回原始数据和交互项数据的拼接
        return pd.concat([X, interaction_data], axis=1)

# Step 5:回归与输出
preprocessor = ColumnTransformer(
    transformers=[
        ('custom_poly', CustomPolynomialFeatures(degree=2), poly_features),
        ('passthrough', 'passthrough', non_poly_features)
    ])

# 确定模型
model=Lasso(alpha=1.0)
# 创建 pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  
    ('scaler', StandardScaler()),
    ('model', model)
])


# 训练模型
pipeline.fit(X_train, y_train)

# 预测测试集
test_predictions = pipeline.predict(test_data)

# 将预测结果保存到 DataFrame 中
test_data['Price'] = test_predictions
test_data = test_data[['Price']]

# 保存到 CSV 文件
output_file = f"D:/期中考试/project/prediction_lasso.csv"
test_data.reset_index(drop=True, inplace=True)
test_data.to_csv(output_file, index=True)
print(f"预测结果已保存到 {output_file}")

In [ ]:
###Ridge
# Step 1: 使用 pandas 读取 excel 文件
train_data = pd.read_excel("D:/期中考试/project/train_v1.xlsx") 
test_data = pd.read_excel("D:/期中考试/project/test_v1.xlsx") 
details_data=pd.read_excel("D:/期中考试/project/details_v1.xlsx")
# Step 2: 数据处理
# Step 2.1: 保留核心变量
train_data['is_train'] = 1  # 标记训练集
test_data['is_train'] = 0   # 标记测试集
combined_data = pd.concat([train_data, test_data], axis=0)
combined_data = combined_data[['ID','价格', '城市', '区域','板块','小区名称','环线', '所在楼层','房屋户型', '梯户比例', 
'配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限','建筑面积', '套内面积','房屋朝向','is_train','lon','lat']]
details_data=pd.read_excel("D:/期中考试/project/details_v1.xlsx")
details_data=details_data[['板块','名称','容 积 率','绿化率']]

# 根据小区名称和名称进行匹配（左连接）
combined_data = combined_data.merge(details_data, left_on=['板块','小区名称'], right_on=['板块','名称'], how='left')
combined_data.drop(columns=['名称'], inplace=True) 
combined_data['小区名称'] = combined_data['小区名称'] + combined_data['板块'].astype(str)

# Step 2.2: 生成新变量-对数化面积（加1是为了克服部分为0的数据）、把小区变量进行变换，克服共线性问题
#combined_data['价格']=combined_data['价格']/combined_data['建筑面积']
combined_data['套内面积'] = combined_data['套内面积'].replace(0, combined_data['套内面积'].mean())
combined_data['套内面积比'] = combined_data['套内面积'] / combined_data['建筑面积']
combined_data['建筑面积_对数'] = np.log(combined_data['建筑面积'] + 1)
combined_data['套内面积_对数'] = np.log(combined_data['套内面积'] + 1)
combined_data['建筑面积_平方'] = combined_data['建筑面积'] ** 2
combined_data['套内面积_平方'] = combined_data['套内面积'] ** 2
combined_data['绿化率']=combined_data['绿化率']*combined_data['建筑面积']/100
combined_data['容 积 率']=combined_data['容 积 率']*combined_data['建筑面积']
combined_data['面积级别'] = (combined_data['建筑面积'] / 35 + 1).astype(int)

# Step 2.3:转换变量-房屋户型（分为5个变量）
combined_data['室'] = combined_data['房屋户型'].str.extract(r'(\d+)室').astype(float)
combined_data['厅'] = combined_data['房屋户型'].str.extract(r'(\d+)厅').astype(float)
combined_data['厨'] = combined_data['房屋户型'].str.extract(r'(\d+)厨').astype(float)
combined_data['卫'] = combined_data['房屋户型'].str.extract(r'(\d+)卫').astype(float)
combined_data['房间'] = combined_data['房屋户型'].str.extract(r'(\d+)房间').astype(float)
combined_data=combined_data.fillna(0)
combined_data = combined_data.drop('房屋户型', axis=1)
#转为int，便于后续根据是否为0来进行填充
combined_data['室'] = combined_data['室'].astype(int)
combined_data['厅'] = combined_data['厅'].astype(int)
combined_data['厨'] = combined_data['厨'].astype(int)
combined_data['卫'] = combined_data['卫'].astype(int).replace(0, 1)
combined_data['房间'] = combined_data['房间'].astype(int)

# 如果房间数目为 0，则将室和厅的值相加填入房间
combined_data['房间'] = combined_data.apply(lambda row: row['室'] + row['厅'] if row['房间'] == 0 else row['房间'], axis=1)
combined_data['室卫比'] = combined_data['房间'] / combined_data['卫'].replace(0, 1)

# Step 2.4: 转换变量-所在楼层（分为楼层类型和总层数）
def extract_floor_info(floor_str):
    # 提取楼层类型（如“中楼层”）
    floor_type = re.match(r'(.+) \(', floor_str).group(1)
    # 提取总层数（如“25”）
    total_floors = int(re.search(r'共(\d+)层', floor_str).group(1))
    return floor_type, total_floors
combined_data['楼层类型'] = combined_data['所在楼层'].apply(lambda x: extract_floor_info(x)[0])
combined_data['总层数'] = combined_data['所在楼层'].apply(lambda x: extract_floor_info(x)[1])
combined_data.drop(columns=['所在楼层'], inplace=True)
# Step 2.5: 转换变量-梯户比例（从"X梯X户"转换为数值型变量）
def convert_ratio(ratio_str):
    # 汉字数字映射
    chinese_to_num = {
    '一': 1, '二': 2, '两': 2, '三': 3, '四': 4, '五': 5,'六': 6, '七': 7, '八': 8, '九': 9, '十': 10,
    '十一': 11, '十二': 12, '十三': 13, '十四': 14, '十五': 15,'十六': 16, '十七': 17, '十八': 18, '十九': 19, '二十': 20,
    '二十一': 21, '二十二': 22, '二十三': 23, '二十四': 24, '二十五': 25,'二十六': 26, '二十七': 27, '二十八': 28, '二十九': 29, '三十': 30,
    '三十一': 31, '三十二': 32, '三十三': 33, '三十四': 34, '三十五': 35,'三十六': 36, '三十七': 37, '三十八': 38, '三十九': 39, '四十': 40,
    '四十一': 41, '四十二': 42, '四十三': 43, '四十四': 44, '四十五': 45,'四十六': 46, '四十七': 47, '四十八': 48, '四十九': 49, '五十': 50,
    '五十一': 51, '五十二': 52, '五十三': 53, '五十四': 54, '五十五': 55,'五十六': 56, '五十七': 57, '五十八': 58, '五十九': 59, '六十': 60,
    '六十一': 61, '六十二': 62, '六十三': 63, '六十四': 64, '六十五': 65,'六十六': 66, '六十七': 67, '六十八': 68, '六十九': 69, '七十': 70,
    '七十一': 71, '七十二': 72, '七十三': 73, '七十四': 74, '七十五': 75,'七十六': 76, '七十七': 77, '七十八': 78, '七十九': 79, '八十': 80,
    '八十一': 81, '八十二': 82, '八十三': 83, '八十四': 84, '八十五': 85,'八十六': 86, '八十七': 87, '八十八': 88, '八十九': 89, '九十': 90,
    '九十一': 91, '九十二': 92, '九十三': 93, '九十四': 94, '九十五': 95,'九十六': 96, '九十七': 97, '九十八': 98, '九十九': 99, '一百': 100
    }
     # 如果输入是 '0'，直接返回 0
    if ratio_str == '0':
        return 0
    # 提取梯数和户数
    parts = ratio_str.split('梯')
    if len(parts) != 2:
        raise ValueError(f"Invalid format: {ratio_str}")
    elevators = parts[0].strip()
    households = parts[1].replace('户', '').strip()
    # 转换为阿拉伯数字
    elevators_num = chinese_to_num.get(elevators, elevators)
    households_num = chinese_to_num.get(households, households)
    # 计算比例
    try:
        ratio = int(households_num) / int(elevators_num)
    except ZeroDivisionError:
        ratio = np.nan  # 如果梯数为0，返回NaN
    return ratio
combined_data['梯户比例'] = combined_data['梯户比例'].astype(str)
combined_data['梯户比例'] = combined_data['梯户比例'].apply(convert_ratio)


# Step 2.6: 生成新变量-虚拟变量（本质为固定效应）
#combined_data = pd.get_dummies(combined_data, columns=['城市', '区域','板块','小区名称','环线', '楼层类型','配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限'], drop_first=True)
combined_data = pd.get_dummies(combined_data, columns=['城市', '区域','板块','小区名称','环线', '楼层类型','配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限','面积级别'], drop_first=True)


# Step 2.7:生成新变量-小区*面积的交乘项（主因在于，小区本身对应的虚拟变量有3000+个，若直接将其通过polynomial函数处理，会导致内存不足）
neighborhood_columns = [col for col in combined_data.columns if col.startswith('板块_')]
for col in neighborhood_columns:
    # 与建筑面积的交乘项
    new_col_name = f'{col}_建筑面积'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积']

    new_col_name = f'{col}_建筑面积_对数'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积_对数']

    new_col_name = f'{col}_建筑面积_平方'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积_平方']

# Step 2.8:基本朝向
combined_data['朝南'] = combined_data['房屋朝向'].str.contains('南', na=False).astype(int)
combined_data['朝北'] = combined_data['房屋朝向'].str.contains('北', na=False).astype(int)
combined_data['朝东'] = combined_data['房屋朝向'].str.contains('东', na=False).astype(int)
combined_data['朝西'] = combined_data['房屋朝向'].str.contains('西', na=False).astype(int)
combined_data=combined_data.drop(columns=['房屋朝向'])
# 复合特征
combined_data['朝向数量'] = combined_data[['朝南', '朝北', '朝东', '朝西']].sum(axis=1)
combined_data['南北通透'] = ((combined_data['朝南'] == 1) & (combined_data['朝北'] == 1)).astype(int)
combined_data['到市中心距离'] = np.sqrt((combined_data['lon'] - 116.4) ** 2 + (combined_data['lat'] - 39.9) ** 2)
combined_data=combined_data.drop(columns=['lon','lat'])

# Step 3: 划分训练集和验证集
train_data = combined_data[combined_data['is_train'] == 1].drop(columns=['is_train'])
test_data = combined_data[combined_data['is_train'] == 0].drop(columns=['is_train'])
X_train = train_data.drop(columns=['价格','ID'])  # 自变量
y_train = train_data['价格']  # 因变量
# 按照训练集的列顺序对齐测试集
test_data = test_data[X_train.columns]


# Step 4: 创建 ColumnTransformer 来处理特征
non_poly_features = [col for col in X_train.columns if (col.startswith('板块_') or col.startswith('小区名称_') or col.startswith('面积级别'))]
poly_features = [col for col in X_train.columns if not (col.startswith('板块_') or col.startswith('小区名称_') or col.startswith('面积级别'))]

class CustomPolynomialFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2):
        self.degree = degree

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # 获取所有列名
        columns = X.columns.tolist()
        # 初始化交互项列表
        interaction_terms = []
        # 遍历所有列名，生成交互项
        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                # 检查是否属于不同类别
                if columns[i][:2] != columns[j][:2]:
                    interaction_terms.append((columns[i], columns[j]))
        # 生成交互项数据
        interaction_data = pd.DataFrame()
        for term in interaction_terms:
            interaction_data[f'{term[0]}_x_{term[1]}'] = X[term[0]] * X[term[1]]
        # 返回原始数据和交互项数据的拼接
        return pd.concat([X, interaction_data], axis=1)

# Step 5:回归与输出
preprocessor = ColumnTransformer(
    transformers=[
        ('custom_poly', CustomPolynomialFeatures(degree=2), poly_features),
        ('passthrough', 'passthrough', non_poly_features)
    ])

# 确定模型
model=Ridge(alpha=1000)
# 创建 pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  
    ('scaler', StandardScaler()),
    ('model', model)
])


# 训练模型
pipeline.fit(X_train, y_train)

# 预测测试集
test_predictions = pipeline.predict(test_data)

# 将预测结果保存到 DataFrame 中
test_data['Price'] = test_predictions
test_data = test_data[['Price']]

# 保存到 CSV 文件
output_file = f"D:/期中考试/project/prediction_ridge.csv"
test_data.reset_index(drop=True, inplace=True)
test_data.to_csv(output_file, index=True)
print(f"预测结果已保存到 {output_file}")

In [24]:
###ElasticNet
# Step 1: 使用 pandas 读取 excel 文件
train_data = pd.read_excel("D:/期中考试/project/train_v1.xlsx") 
test_data = pd.read_excel("D:/期中考试/project/test_v1.xlsx") 
details_data=pd.read_excel("D:/期中考试/project/details_v1.xlsx")
# Step 2: 数据处理
# Step 2.1: 保留核心变量
train_data['is_train'] = 1  # 标记训练集
test_data['is_train'] = 0   # 标记测试集
combined_data = pd.concat([train_data, test_data], axis=0)
combined_data = combined_data[['ID','价格', '城市', '区域','板块','小区名称','环线', '所在楼层','房屋户型', '梯户比例', 
'配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限','建筑面积', '套内面积','房屋朝向','is_train','lon','lat']]
details_data=pd.read_excel("D:/期中考试/project/details_v1.xlsx")
details_data=details_data[['板块','名称','容 积 率','绿化率']]

# 根据小区名称和名称进行匹配（左连接）
combined_data = combined_data.merge(details_data, left_on=['板块','小区名称'], right_on=['板块','名称'], how='left')
combined_data.drop(columns=['名称'], inplace=True) 
combined_data['小区名称'] = combined_data['小区名称'] + combined_data['板块'].astype(str)

# Step 2.2: 生成新变量-对数化面积（加1是为了克服部分为0的数据）、把小区变量进行变换，克服共线性问题
#combined_data['价格']=combined_data['价格']/combined_data['建筑面积']
combined_data['套内面积'] = combined_data['套内面积'].replace(0, combined_data['套内面积'].mean())
combined_data['套内面积比'] = combined_data['套内面积'] / combined_data['建筑面积']
combined_data['建筑面积_对数'] = np.log(combined_data['建筑面积'] + 1)
combined_data['套内面积_对数'] = np.log(combined_data['套内面积'] + 1)
combined_data['建筑面积_平方'] = combined_data['建筑面积'] ** 2
combined_data['套内面积_平方'] = combined_data['套内面积'] ** 2
combined_data['绿化率']=combined_data['绿化率']*combined_data['建筑面积']/100
combined_data['容 积 率']=combined_data['容 积 率']*combined_data['建筑面积']
combined_data['面积级别'] = (combined_data['建筑面积'] / 35 + 1).astype(int)

# Step 2.3:转换变量-房屋户型（分为5个变量）
combined_data['室'] = combined_data['房屋户型'].str.extract(r'(\d+)室').astype(float)
combined_data['厅'] = combined_data['房屋户型'].str.extract(r'(\d+)厅').astype(float)
combined_data['厨'] = combined_data['房屋户型'].str.extract(r'(\d+)厨').astype(float)
combined_data['卫'] = combined_data['房屋户型'].str.extract(r'(\d+)卫').astype(float)
combined_data['房间'] = combined_data['房屋户型'].str.extract(r'(\d+)房间').astype(float)
combined_data=combined_data.fillna(0)
combined_data = combined_data.drop('房屋户型', axis=1)
#转为int，便于后续根据是否为0来进行填充
combined_data['室'] = combined_data['室'].astype(int)
combined_data['厅'] = combined_data['厅'].astype(int)
combined_data['厨'] = combined_data['厨'].astype(int)
combined_data['卫'] = combined_data['卫'].astype(int).replace(0, 1)
combined_data['房间'] = combined_data['房间'].astype(int)

# 如果房间数目为 0，则将室和厅的值相加填入房间
combined_data['房间'] = combined_data.apply(lambda row: row['室'] + row['厅'] if row['房间'] == 0 else row['房间'], axis=1)
combined_data['室卫比'] = combined_data['房间'] / combined_data['卫'].replace(0, 1)

# Step 2.4: 转换变量-所在楼层（分为楼层类型和总层数）
def extract_floor_info(floor_str):
    # 提取楼层类型（如“中楼层”）
    floor_type = re.match(r'(.+) \(', floor_str).group(1)
    # 提取总层数（如“25”）
    total_floors = int(re.search(r'共(\d+)层', floor_str).group(1))
    return floor_type, total_floors
combined_data['楼层类型'] = combined_data['所在楼层'].apply(lambda x: extract_floor_info(x)[0])
combined_data['总层数'] = combined_data['所在楼层'].apply(lambda x: extract_floor_info(x)[1])
combined_data.drop(columns=['所在楼层'], inplace=True)
# Step 2.5: 转换变量-梯户比例（从"X梯X户"转换为数值型变量）
def convert_ratio(ratio_str):
    # 汉字数字映射
    chinese_to_num = {
    '一': 1, '二': 2, '两': 2, '三': 3, '四': 4, '五': 5,'六': 6, '七': 7, '八': 8, '九': 9, '十': 10,
    '十一': 11, '十二': 12, '十三': 13, '十四': 14, '十五': 15,'十六': 16, '十七': 17, '十八': 18, '十九': 19, '二十': 20,
    '二十一': 21, '二十二': 22, '二十三': 23, '二十四': 24, '二十五': 25,'二十六': 26, '二十七': 27, '二十八': 28, '二十九': 29, '三十': 30,
    '三十一': 31, '三十二': 32, '三十三': 33, '三十四': 34, '三十五': 35,'三十六': 36, '三十七': 37, '三十八': 38, '三十九': 39, '四十': 40,
    '四十一': 41, '四十二': 42, '四十三': 43, '四十四': 44, '四十五': 45,'四十六': 46, '四十七': 47, '四十八': 48, '四十九': 49, '五十': 50,
    '五十一': 51, '五十二': 52, '五十三': 53, '五十四': 54, '五十五': 55,'五十六': 56, '五十七': 57, '五十八': 58, '五十九': 59, '六十': 60,
    '六十一': 61, '六十二': 62, '六十三': 63, '六十四': 64, '六十五': 65,'六十六': 66, '六十七': 67, '六十八': 68, '六十九': 69, '七十': 70,
    '七十一': 71, '七十二': 72, '七十三': 73, '七十四': 74, '七十五': 75,'七十六': 76, '七十七': 77, '七十八': 78, '七十九': 79, '八十': 80,
    '八十一': 81, '八十二': 82, '八十三': 83, '八十四': 84, '八十五': 85,'八十六': 86, '八十七': 87, '八十八': 88, '八十九': 89, '九十': 90,
    '九十一': 91, '九十二': 92, '九十三': 93, '九十四': 94, '九十五': 95,'九十六': 96, '九十七': 97, '九十八': 98, '九十九': 99, '一百': 100
    }
     # 如果输入是 '0'，直接返回 0
    if ratio_str == '0':
        return 0
    # 提取梯数和户数
    parts = ratio_str.split('梯')
    if len(parts) != 2:
        raise ValueError(f"Invalid format: {ratio_str}")
    elevators = parts[0].strip()
    households = parts[1].replace('户', '').strip()
    # 转换为阿拉伯数字
    elevators_num = chinese_to_num.get(elevators, elevators)
    households_num = chinese_to_num.get(households, households)
    # 计算比例
    try:
        ratio = int(households_num) / int(elevators_num)
    except ZeroDivisionError:
        ratio = np.nan  # 如果梯数为0，返回NaN
    return ratio
combined_data['梯户比例'] = combined_data['梯户比例'].astype(str)
combined_data['梯户比例'] = combined_data['梯户比例'].apply(convert_ratio)


# Step 2.6: 生成新变量-虚拟变量（本质为固定效应）
#combined_data = pd.get_dummies(combined_data, columns=['城市', '区域','板块','小区名称','环线', '楼层类型','配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限'], drop_first=True)
combined_data = pd.get_dummies(combined_data, columns=['城市', '区域','板块','小区名称','环线', '楼层类型','配备电梯', '建筑结构', '装修情况','交易权属','房屋用途','房屋年限','面积级别'], drop_first=True)


# Step 2.7:生成新变量-小区*面积的交乘项（主因在于，小区本身对应的虚拟变量有3000+个，若直接将其通过polynomial函数处理，会导致内存不足）
neighborhood_columns = [col for col in combined_data.columns if col.startswith('板块_')]
for col in neighborhood_columns:
    # 与建筑面积的交乘项
    new_col_name = f'{col}_建筑面积'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积']

    new_col_name = f'{col}_建筑面积_对数'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积_对数']

    new_col_name = f'{col}_建筑面积_平方'
    combined_data[new_col_name] = combined_data[col] * combined_data['建筑面积_平方']

# Step 2.8:基本朝向
combined_data['朝南'] = combined_data['房屋朝向'].str.contains('南', na=False).astype(int)
combined_data['朝北'] = combined_data['房屋朝向'].str.contains('北', na=False).astype(int)
combined_data['朝东'] = combined_data['房屋朝向'].str.contains('东', na=False).astype(int)
combined_data['朝西'] = combined_data['房屋朝向'].str.contains('西', na=False).astype(int)
combined_data=combined_data.drop(columns=['房屋朝向'])
# 复合特征
combined_data['朝向数量'] = combined_data[['朝南', '朝北', '朝东', '朝西']].sum(axis=1)
combined_data['南北通透'] = ((combined_data['朝南'] == 1) & (combined_data['朝北'] == 1)).astype(int)
combined_data['到市中心距离'] = np.sqrt((combined_data['lon'] - 116.4) ** 2 + (combined_data['lat'] - 39.9) ** 2)
combined_data=combined_data.drop(columns=['lon','lat'])

# Step 3: 划分训练集和验证集
train_data = combined_data[combined_data['is_train'] == 1].drop(columns=['is_train'])
test_data = combined_data[combined_data['is_train'] == 0].drop(columns=['is_train'])
X_train = train_data.drop(columns=['价格','ID'])  # 自变量
y_train = train_data['价格']  # 因变量
# 按照训练集的列顺序对齐测试集
test_data = test_data[X_train.columns]


# Step 4: 创建 ColumnTransformer 来处理特征
non_poly_features = [col for col in X_train.columns if (col.startswith('板块_') or col.startswith('小区名称_') or col.startswith('面积级别'))]
poly_features = [col for col in X_train.columns if not (col.startswith('板块_') or col.startswith('小区名称_') or col.startswith('面积级别'))]

class CustomPolynomialFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, degree=2):
        self.degree = degree

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # 获取所有列名
        columns = X.columns.tolist()
        # 初始化交互项列表
        interaction_terms = []
        # 遍历所有列名，生成交互项
        for i in range(len(columns)):
            for j in range(i + 1, len(columns)):
                # 检查是否属于不同类别
                if columns[i][:2] != columns[j][:2]:
                    interaction_terms.append((columns[i], columns[j]))
        # 生成交互项数据
        interaction_data = pd.DataFrame()
        for term in interaction_terms:
            interaction_data[f'{term[0]}_x_{term[1]}'] = X[term[0]] * X[term[1]]
        # 返回原始数据和交互项数据的拼接
        return pd.concat([X, interaction_data], axis=1)

# Step 5:回归与输出
preprocessor = ColumnTransformer(
    transformers=[
        ('custom_poly', CustomPolynomialFeatures(degree=2), poly_features),
        ('passthrough', 'passthrough', non_poly_features)
    ])

# 确定模型
model=ElasticNet(alpha=0.1, l1_ratio=0.5)
#model=Lasso(alpha=1.0)
# 创建 pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),  
    ('scaler', StandardScaler()),
    ('model', model)
])


# 训练模型
pipeline.fit(X_train, y_train)

# 预测测试集
test_predictions = pipeline.predict(test_data)

# 将预测结果保存到 DataFrame 中
test_data['Price'] = test_predictions
test_data = test_data[['Price']]

# 保存到 CSV 文件
output_file = f"D:/期中考试/project/prediction_elsticnet.csv"
test_data.reset_index(drop=True, inplace=True)
test_data.to_csv(output_file, index=True)
print(f"预测结果已保存到 {output_file}")

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6421352469016786.0, tolerance: 58634969843450.805
  model = cd_fast.enet_coordinate_descent(


预测结果已保存到 D:/期中考试/project/prediction_new_v11.csv
